In [2]:
import numpy as np
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset

2024-11-07 22:31:27.258337: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-07 22:31:27.407363: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730998887.461373    6649 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730998887.476905    6649 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-07 22:31:27.609221: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
train = pd.read_csv('./train(1).csv')
test = pd.read_csv('./test(1).csv')

In [4]:
df = pd.concat([train,test])
df = df.dropna()

In [5]:
categories = set(df['category']) 
subcategories = set(df['sub_category'])

In [6]:
len(categories), len(subcategories)

(12, 38)

In [7]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

/home/hamza/miniconda3/envs/condaenv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
from sklearn.preprocessing import LabelEncoder

category_encoder = LabelEncoder()
df['category_encoded'] = category_encoder.fit_transform(df['category'])

subcategory_encoder = LabelEncoder()
df['subcategory_encoded'] = subcategory_encoder.fit_transform(df['sub_category'])

In [9]:
category_dict = df.groupby('category')['sub_category'].apply(lambda x: list(set(x))).to_dict() 
category_dict

{'Any Other Cyber Crime': ['Other'],
 'Crime Against Women & Children': ['Cyber Blackmailing & Threatening',
  'Sexual Harassment',
  'Computer Generated CSAM/CSEM'],
 'Cryptocurrency Crime': ['Cryptocurrency Fraud'],
 'Cyber Attack/ Dependent Crimes': ['Malware Attack',
  'Ransomware Attack',
  'Denial of Service (DoS)/Distributed Denial of Service (DDOS) attacks',
  'SQL Injection',
  'Data Breach/Theft',
  'Tampering with computer source documents',
  'Hacking/Defacement'],
 'Cyber Terrorism': ['Cyber Terrorism'],
 'Hacking  Damage to computercomputer system etc': ['Unauthorised AccessData Breach',
  'Email Hacking',
  'Website DefacementHacking',
  'Damage to computer computer systems etc',
  'Tampering with computer source documents'],
 'Online Cyber Trafficking': ['Online Trafficking'],
 'Online Financial Fraud': ['Internet Banking Related Fraud',
  'DematDepository Fraud',
  'EWallet Related Fraud',
  'Business Email CompromiseEmail Takeover',
  'UPI Related Frauds',
  'Fraud Ca

In [10]:
subcategory_to_category = {sub: cat for cat, subs in category_dict.items() for sub in subs} 
subcategory_to_category

{'Other': 'Any Other Cyber Crime',
 'Cyber Blackmailing & Threatening': 'Crime Against Women & Children',
 'Sexual Harassment': 'Crime Against Women & Children',
 'Computer Generated CSAM/CSEM': 'Crime Against Women & Children',
 'Cryptocurrency Fraud': 'Cryptocurrency Crime',
 'Malware Attack': 'Cyber Attack/ Dependent Crimes',
 'Ransomware Attack': 'Cyber Attack/ Dependent Crimes',
 'Denial of Service (DoS)/Distributed Denial of Service (DDOS) attacks': 'Cyber Attack/ Dependent Crimes',
 'SQL Injection': 'Cyber Attack/ Dependent Crimes',
 'Data Breach/Theft': 'Cyber Attack/ Dependent Crimes',
 'Tampering with computer source documents': 'Hacking  Damage to computercomputer system etc',
 'Hacking/Defacement': 'Cyber Attack/ Dependent Crimes',
 'Cyber Terrorism': 'Cyber Terrorism',
 'Unauthorised AccessData Breach': 'Hacking  Damage to computercomputer system etc',
 'Email Hacking': 'Hacking  Damage to computercomputer system etc',
 'Website DefacementHacking': 'Hacking  Damage to comp

In [11]:
df.head()


,category,sub_category,crimeaditionalinfo,category_encoded,subcategory_encoded
0,Online and Social Media Related Crime,Cyber Bullying Stalking Sexting,I had continue received random calls and abusi...,9,6
1,Online Financial Fraud,Fraud CallVishing,The above fraudster is continuously messaging ...,7,17
2,Online Gambling Betting,Online Gambling Betting,He is acting like a police and demanding for m...,8,23
3,Online and Social Media Related Crime,Online Job Fraud,In apna Job I have applied for job interview f...,9,24
4,Online Financial Fraud,Fraud CallVishing,I received a call from lady stating that she w...,7,17


In [12]:
train_size = 0.75 # train:test ratio of 3:1
split_index = int(train_size*len(df))
train, test = df[:split_index], df[split_index:]

In [13]:
train.head()

,category,sub_category,crimeaditionalinfo,category_encoded,subcategory_encoded
0,Online and Social Media Related Crime,Cyber Bullying Stalking Sexting,I had continue received random calls and abusi...,9,6
1,Online Financial Fraud,Fraud CallVishing,The above fraudster is continuously messaging ...,7,17
2,Online Gambling Betting,Online Gambling Betting,He is acting like a police and demanding for m...,8,23
3,Online and Social Media Related Crime,Online Job Fraud,In apna Job I have applied for job interview f...,9,24
4,Online Financial Fraud,Fraud CallVishing,I received a call from lady stating that she w...,7,17


In [14]:
x_train, x_test = train['crimeaditionalinfo'].values, test['crimeaditionalinfo'].values
y_train, y_test = train['subcategory_encoded'].values, test['subcategory_encoded'].values

In [15]:
train_encoded = tokenizer(list(x_train), padding=True, truncation=True, max_length=512)
test_encoded = tokenizer(list(x_test), padding=True, truncation=True, max_length=512)

In [16]:
import torch
from torch.utils.data import Dataset

# Define a custom dataset class to handle tokenized data and labels
class ComplaintDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # Retrieve encoded input and convert to tensors
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])  # Convert labels to tensors
        return item

    def __len__(self):
        return len(self.labels)

# Convert your labels into torch tensors
train_labels = torch.tensor(y_train)
test_labels = torch.tensor(y_test)

# Create the dataset
train_dataset = ComplaintDataset(train_encoded, train_labels)
test_dataset = ComplaintDataset(test_encoded, test_labels)


In [17]:
# Load pre-trained mBERT model for sequence classification
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-multilingual-cased", num_labels=len(subcategories))
model.config.num_parameters = 135333131

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,  # Set a smaller batch size
    per_device_eval_batch_size=64,   # Evaluation batch size
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=200,
    gradient_accumulation_steps=8,   # Accumulate gradients over 8 steps
    use_cpu=False
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
import os
os.environ["WANDB_MODE"] = "disabled"

In [ ]:
# Initialize the Trainer
trainer = Trainer(
    model=model,                         # The model to be trained
    args=training_args,                  # Training arguments
    train_dataset=train_dataset,         # Training dataset
    eval_dataset=test_dataset,           # Evaluation dataset
)

# Fine-tune the model
trainer.train()

/tmp/ipykernel_6649/3379088383.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])  # Convert labels to tensors


Epoch,Training Loss,Validation Loss


In [ ]:
torch.save(model.state_dict(),'model.pth')

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
import numpy as np

# Make predictions on the test dataset
predicted_labels = np.argmax(trainer.predict(test_dataset))  # Get the predicted subcategory labels

# Evaluate metrics for subcategory level
subcategory_accuracy = accuracy_score(y_test, predicted_labels)
subcategory_precision = precision_score(y_test, predicted_labels, average="weighted")
subcategory_recall = recall_score(y_test, predicted_labels, average="weighted")

print(f"Subcategory Level - Accuracy: {subcategory_accuracy:.4f}, Precision: {subcategory_precision:.4f}, Recall: {subcategory_recall:.4f}")

# Map the true and predicted subcategories to their parent categories
true_categories = [subcategory_to_category[subcat] for subcat in y_test]
predicted_categories = [subcategory_to_category[subcat] for subcat in predicted_labels]

# Encode the true and predicted categories as integers for calculating metrics
unique_categories = list(set(true_categories + predicted_categories))
category_to_index = {cat: idx for idx, cat in enumerate(unique_categories)}

true_category_indices = [category_to_index[cat] for cat in true_categories]
predicted_category_indices = [category_to_index[cat] for cat in predicted_categories]

# Evaluate metrics for category level
category_accuracy = accuracy_score(true_category_indices, predicted_category_indices)
category_precision = precision_score(true_category_indices, predicted_category_indices, average="weighted")
category_recall = recall_score(true_category_indices, predicted_category_indices, average="weighted")

print(f"Category Level - Accuracy: {category_accuracy:.4f}, Precision: {category_precision:.4f}, Recall: {category_recall:.4f}")
